In [1]:
import pandas as pd
import os 
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

os.chdir('C:/Users/George/Documents/Baseball/2008-2017_pitch_data')

In [2]:
rawData = pd.read_csv('baseballdata1517.csv')
rawData = rawData.drop(['Unnamed: 0'], axis=1)

In [3]:
randomPitchers = list(pd.Series(rawData['pitcher'].unique()).sample(10))

In [4]:
data = rawData[rawData['pitcher'].isin(randomPitchers)]

In [5]:
#Binary on base not on a base
data[['on_3b', 'on_2b', 'on_1b']] = data[['on_3b', 'on_2b', 'on_1b']].isna() == False

C:\Users\George\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [6]:
#adding indices 
data['game_date_index'] = data['game_date'].apply(lambda a: a.replace('-', ''))
data['game_index'] = data['game_date_index'] + '_' + data.away_team + '_' + data.home_team 
data['pitcher_apperance_index'] = data['game_date_index'] + '_' + data['pitcher'].astype(str)    
data['atbat_id'] = data['pitcher_apperance_index'] + "_" + data['at_bat_number'].astype(str)

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [7]:
hit_list = ['double', 'single', 'home_run', 'triple']
data2 = pd.DataFrame()
for i in data['pitcher_apperance_index'].unique():
    
    #selecing single game
    apperance = data[data['pitcher_apperance_index'] == i] 
    apperance = apperance.sort_values(['at_bat_number','pitch_number']) #sorting at-bats and pitches
    
    #pitch count
    apperance['pitch_count_game'] = np.arange(len(apperance)) 
    
    #Number of walks
    apperance['number_of_walks'] = (apperance['events'] == 'walk').cumsum().shift(1).fillna(0)
    
    #Number of hits
    apperance['number_of_hits'] = apperance['events'].isin(hit_list).cumsum().shift(1).fillna(0)
    
    data2 = data2.append(apperance)

In [8]:
data3 = pd.DataFrame()
for i in data2['atbat_id'].unique():
    atbat = data2[data2['atbat_id'] == i]
    atbat = atbat.sort_values('pitch_number')

    
    atbat['last_pitch'] = atbat['pitch_type'].shift(1).fillna('NewAB')

    atbat[['last_plate_x', 'last_plate_z']] = atbat[['plate_x', 'plate_z']].shift(1)
    atbat[['last_2_plate_x', 'last_2_plate_z']] = atbat[['plate_x', 'plate_z']].shift(1).rolling(2).mean()
    
    atbat['last_pitch_speed'] = atbat['effective_speed'].shift(1)
    atbat['last_2_pitch_speed'] = atbat['effective_speed'].shift(1).rolling(2).mean()
    
    data3 = data3.append(atbat)    

In [9]:
conditions = [
    (data3['on_1b'] == True) & (data3['on_2b'] == True) & (data3['on_3b'] == True),
    (data3['on_2b'] == True) | (data3['on_3b'] == True),
    (data3['on_1b'] == True)
]

status = ['BL', 'RISP', '1B']

data3['Runners_On'] = np.select(conditions, status, default='BasesEmpty')

In [10]:
data3['Fav_Pitcher_Matchup'] = data3['p_throws'] == data3['stand']

In [14]:
toOneHot = np.array(data3[['Runners_On', 'last_pitch']])
onehotencoder = OneHotEncoder(sparse=False) 
oneHotData = onehotencoder.fit_transform(toOneHot)
oneHotData = pd.DataFrame(oneHotData, columns=onehotencoder.get_feature_names(['Runners_On', 'last_pitch']))
data3 = data3.reset_index(drop=True)
data3 = pd.concat([data3, oneHotData], axis=1)

In [98]:
data3['pitch_type'].fillna('FF', inplace = True)

In [99]:
# Label encoding Pitch Type so can be fed into models
labelencoder = LabelEncoder()
PitchTypeEncoded = labelencoder.fit_transform(data3['pitch_type'])

data3['pitch_type_le'] = PitchTypeEncoded

In [285]:
selected_pitcher = randomPitchers[4]
single_pitcher = data3[data3['pitcher'] == selected_pitcher]

In [286]:
single_pitcher['game_date'] = pd.to_datetime(single_pitcher.game_date)
#single_pitcher = single_pitcher[single_pitcher['game_date'] > '2016-01-01']


C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [287]:
pitch_columns = lambda a : "last_pitch_{}".format(a)

last_pitch_columns = pd.Series(single_pitcher['pitch_type'].unique()).apply(pitch_columns)
last_pitch_columns = list(last_pitch_columns)

In [288]:
StandardPredVars = ['strikes', 'balls','pitch_count_game','Runners_On_RISP', 'Fav_Pitcher_Matchup']
PredVars = StandardPredVars + last_pitch_columns
#PredVars.remove('last_pitch_nan')

In [289]:
X = single_pitcher[PredVars]
y = single_pitcher['pitch_type_le'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)

In [290]:
for i in range(2,16):
    dt_clf = DecisionTreeClassifier(min_samples_split=2,
                                    max_depth=4,
                                    max_features=5,
                                    random_state=23)
    dt_clf.fit(X_train, y_train)
    preds = dt_clf.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} max depth the accuracy is {}%'.format(i, round(acc*100,1)))

For 2 max depth the accuracy is 62.3%
For 3 max depth the accuracy is 62.3%
For 4 max depth the accuracy is 62.3%
For 5 max depth the accuracy is 62.3%
For 6 max depth the accuracy is 62.3%
For 7 max depth the accuracy is 62.3%
For 8 max depth the accuracy is 62.3%
For 9 max depth the accuracy is 62.3%
For 10 max depth the accuracy is 62.3%
For 11 max depth the accuracy is 62.3%
For 12 max depth the accuracy is 62.3%
For 13 max depth the accuracy is 62.3%
For 14 max depth the accuracy is 62.3%
For 15 max depth the accuracy is 62.3%


In [291]:
single_pitcher.pitch_type.value_counts() / len(single_pitcher)

FF    0.520571
SL    0.453983
CH    0.024019
IN    0.000951
PO    0.000476
Name: pitch_type, dtype: float64

In [292]:
pd.Series(preds).value_counts() / len(preds)

9    0.545627
3    0.454373
dtype: float64

In [293]:
pd.Series(y_test).value_counts() / len(y_test)

3    0.521863
9    0.458175
0    0.019011
8    0.000951
Name: pitch_type_le, dtype: float64